In [1]:
# Ocean Heat Budget Analyses in the Antarctica continental shelf (WAOM)

# Fabio B Dias - 3 May 2023
# Description:
#     this script compares methods of calculate the OHC and surface heat flux integrated over the continental shelf
#     - update to xarray open_mfdataset from open_dataset brought new functions more efficient to calculate model layer thickness (dz)
# and ocean heat content tendencies (rho0*Cp*dT/dt). 

In [2]:
# read nc output from WAOM 10km run

import xarray as xr
# import pandas as p
import numpy as np
import numpy.ma as ma
import cartopy.crs as ccrs
import matplotlib as mpl
# mpl.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.dates as dates
from matplotlib.ticker import NullFormatter
from matplotlib.colors import LinearSegmentedColormap   # for custom colormaps

from datetime import datetime, timedelta

from netCDF4 import Dataset
from netCDF4 import num2date, date2num
from matplotlib.ticker import NullFormatter
from matplotlib.colors import LinearSegmentedColormap   # for custom colormaps

import gsw

import pyresample

from dask.distributed import Client

import warnings
warnings.filterwarnings('ignore')

In [3]:
fig_path = '/g/data3/hh5/tmp/access-om/fbd581/ROMS/postprocessing/figs/Contour_isobath/'

In [4]:
%%time

# using xr.open_mfdataset

vars2drop = ["ubar","vbar","w","Hsbl","Hbbl","swrad"]

ds = xr.open_mfdataset(paths="/g/data3/hh5/tmp/access-om/fbd581/ROMS/OUTPUT/waom2extend_shflim_S_0.25Q/output_yr5_diag/ocean_avg_00*.nc" , chunks={'eta_rho': '200MB'}, parallel=bool, drop_variables=vars2drop, decode_times=False) # , concat_dim="ocean_time"

#- preserving 5-days avgs
temp = ds.variables["temp"] 
salt = ds.variables["salt"]
shflux = ds.variables["shflux"] 
ssflux = ds.variables["ssflux"]
m = ds.variables["m"]
# HvomT = ds.variables["Hvom_temp"]       ## !!! Huon_temp/Hvom_temp were not saved in the original run
# HuonT = ds.variables["Huon_temp"]       ## now it's running here: /scratch/gi0/fbd581/waom4extend_shflim_S_0.25Q/output_yr10_diag
# Hvom = ds.variables["Hvom"] 
# Huon = ds.variables["Huon"] 

ds = ds.set_coords(['Cs_r', 'Cs_w', 'hc', 'h', 'Vtransform'])

Zo_rho = (ds.hc * ds.s_rho + ds.Cs_r * ds.h) / (ds.hc + ds.h)
z_rho = ds.zeta + (ds.zeta + ds.h) * Zo_rho + ds.zice
print("Vtransform=2")
Zo_w = (ds.hc * ds.s_w + ds.Cs_w * ds.h) / (ds.hc + ds.h)
z_w = ds.zeta + (ds.zeta + ds.h) * Zo_w + ds.zice

ds.close()


Vtransform=2
CPU times: user 10.6 s, sys: 17.5 s, total: 28.1 s
Wall time: 30.2 s


In [13]:
%%time

ds = xr.open_mfdataset(paths="/g/data3/hh5/tmp/access-om/fbd581/ROMS/OUTPUT/waom2extend_shflim_S_0.25Q/output_yr5_diag/ocean_his_00*.nc" , chunks={'eta_rho': '200MB'}, parallel=bool, decode_times=False) #, chuncks="auto", concat_dim="ocean_time"
#- preserving 5-days avgs
temp_snap = ds.temp.drop_isel(ocean_time=[17,36,55])

ds.close()


CPU times: user 7.93 s, sys: 5.95 s, total: 13.9 s
Wall time: 11.9 s


In [9]:
# deleting extra snapshots (with 1.2 days interval): indexes 17, 36 and 55
# temp_snap = xr.DataArray(temp_snap)
# temp_snap = temp_snap.drop_sel(ocean_time=55)
# temp_snap = temp_snap.drop_sel(ocean_time=36)
# temp_snap = temp_snap.drop_sel(ocean_time=17)


In [14]:
temp_snap

<xarray.DataArray 'temp' (ocean_time: 72, s_rho: 31, eta_rho: 2800, xi_rho: 3150)>
dask.array<getitem, shape=(72, 31, 2800, 3150), dtype=float32, chunksize=(8, 31, 64, 3150), chunktype=numpy.ndarray>
Coordinates:
  * s_rho       (s_rho) float64 -0.9839 -0.9516 -0.9194 ... -0.04839 -0.01613
    x_rho       (eta_rho, xi_rho) float64 dask.array<chunksize=(2800, 3150), meta=np.ndarray>
    y_rho       (eta_rho, xi_rho) float64 dask.array<chunksize=(2800, 3150), meta=np.ndarray>
  * ocean_time  (ocean_time) float64 1.266e+08 1.27e+08 ... 1.571e+08 1.576e+08
Dimensions without coordinates: eta_rho, xi_rho
Attributes:
    long_name:  potential temperature
    units:      Celsius
    time:       ocean_time
    field:      temperature, scalar, series

In [ ]:
his = [126576000, 127008000, 127440000, 127872000, 128304000,
    128736000, 129168000, 129600000, 130032000, 130464000, 130896000,
    131328000, 131760000, 132192000, 132624000, 133056000, 133488000,
    133920000, 134028000, 134460000, 134892000, 135324000, 135756000,
    136188000, 136620000, 137052000, 137484000, 137916000, 138348000,
    138780000, 139212000, 139644000, 140076000, 140508000, 140940000,
    141372000, 141804000, 141912000, 142344000, 142776000, 143208000,
    143640000, 144072000, 144504000, 144936000, 145368000, 145800000,
    146232000, 146664000, 147096000, 147528000, 147960000, 148392000,
    148824000, 149256000, 149688000, 149796000, 150228000, 150660000,
    151092000, 151524000, 151956000, 152388000, 152820000, 153252000,
    153684000, 154116000, 154548000, 154980000, 155412000, 155844000,
    156276000, 156708000, 157140000, 157572000]


avg = [126360000, 126792000, 127224000, 127656000, 128088000,
    128520000, 128952000, 129420000, 129852000, 130284000, 130716000,
    131148000, 131580000, 132048000, 132480000, 132912000, 133344000,
    133776000, 134244000, 134676000, 135108000, 135540000, 135972000,
    136404000, 136836000, 137304000, 137736000, 138168000, 138600000,
    139032000, 139464000, 139932000, 140364000, 140796000, 141228000,
    141660000, 142128000, 142560000, 142992000, 143424000, 143856000,
    144288000, 144720000, 145188000, 145620000, 146052000, 146484000,
    146916000, 147348000, 147816000, 148248000, 148680000, 149112000,
    149544000, 150012000, 150444000, 150876000, 151308000, 151740000,
    152172000, 152604000, 153072000, 153504000, 153936000, 154368000,
    154800000, 155232000, 155700000, 156132000, 156564000, 156996000,
    157428000]

print(len(his), len(avg))

In [ ]:
print(temp.shape, temp_snap.shape)


In [ ]:
plt.plot(his, '.b',label='his')
plt.plot(avg, '.r',label='avg')

In [ ]:
# his

his = np.delete(his,17)
his = np.delete(his,36-1)
his = np.delete(his,55-2)

print(len(his))

In [ ]:
plt.plot(his, '.b',label='his')
plt.plot(avg, '.r',label='avg')

In [ ]:
### plt.plot(np.diff(his)/86400)

In [ ]:
temp_snap

In [7]:
temp_snap

<xarray.DataArray (ocean_time: 72, s_rho: 31, eta_rho: 2800, xi_rho: 3150)>
dask.array<getitem, shape=(72, 31, 2800, 3150), dtype=float32, chunksize=(8, 31, 64, 3150), chunktype=numpy.ndarray>
Dimensions without coordinates: ocean_time, s_rho, eta_rho, xi_rho
Attributes:
    long_name:  potential temperature
    units:      Celsius
    time:       ocean_time
    field:      temperature, scalar, series

In [ ]:
%%time

# calculate dT/dt by differentiating temp_snap:
temp_Rate = np.empty(temp_snap.shape)
dT = np.empty(temp_snap.shape)

# needs the initial conditions:
ds = xr.open_dataset('/g/data3/hh5/tmp/access-om/fbd581//ROMS/OUTPUT/waom2extend_shflim_S_0.25Q/output_yr4b/ocean_rst.nc')
temp_ini = ds.variables["temp"].isel(ocean_time=-1, two=0) # 5-days mean
ds.close()

    
tlen = len(temp[:,0,0,0])

In [ ]:
%%time

# transform to DataArray
#temp_snap = xr.DataArray(temp_snap)
temp_ini = xr.DataArray(temp_ini)

# - append temp_ini to first time index in temp_snap and then do diff
temp_snap = xr.concat([temp_ini,temp_snap], 'ocean_time')
dT = temp_snap.diff('ocean_time')
print(dT.shape)

# 
dt = 5*86400 # 5-days in seconds
temp_Rate = np.divide(dT, dt)
temp_Rate = xr.DataArray(temp_Rate)
# temp_Rate=temp_Rate.rename({'dim_0':'ocean_time','dim_1':'s_rho','dim_2':'eta_rho','dim_3':'xi_rho'})

print(dT.shape)

temp_Rate = temp_Rate.transpose('ocean_time','s_rho','eta_rho','xi_rho')

print(temp_Rate.shape)

In [ ]:
# calculate surface sigma_theta (potential density)
sigma_t_sfc = gsw.rho(salt[:,-1,:,:],temp[:,-1,:,:],0) - 1000

# load ice draft to create masks
di = xr.open_dataset('/g/data3/hh5/tmp/access-om/fbd581/ROMS/OUTPUT/waom2extend_shflim_S_0.25Q/output_yr5_diag/ocean_avg_0001.nc')
ice_draft = di.variables["zice"]

mask_zice = ma.masked_where(ice_draft < 0, np.ones(ice_draft.shape))
mask_outice = ma.masked_where(ice_draft >= 0, np.ones(ice_draft.shape))
di.close()

dg = xr.open_dataset("/g/data3/hh5/tmp/access-om/fbd581/ROMS/waom2_frc/waom2extend_grd.nc")

lat_rho = dg.variables["lat_rho"]
lon_rho = dg.variables["lon_rho"]
lat_u = dg.variables["lat_u"]
lon_u = dg.variables["lon_u"]
lat_v = dg.variables["lat_v"]
lon_v = dg.variables["lon_v"]
pm = dg.variables["pm"]
pn = dg.variables["pn"]
h = dg.variables["h"]

ds.coords['lat_rho']=lat_rho.transpose() # put lat_rho into ds dataset
ds.coords['lon_rho']=lon_rho.transpose() # put lon_rho into ds dataset

area=np.divide(1,pm*pn)

In [ ]:
%%time
# method 2 to calculate Dz:
# z_w=z_w.chunks(chunks={'eta_rho': '200MB'}) # couldn't change chunks.

Z_w = z_w.transpose('ocean_time','s_w','eta_rho','xi_rho')
print(z_w.shape, Z_w.shape)
dz = np.diff(Z_w,axis=1)


In [ ]:
# this mask needs to be replaced by mask_shelf2 that uses the 1500-m contour

mask_shelf = ma.masked_where(h > 1500, np.ones(h.shape))

In [ ]:
# determine constants:
rho0 = 1025 # kg. m-3
Cp = 3989.245 # J.kg-1.degC-1
Tf = -1.95 # degC

In [ ]:
%%time

# use -1000 mask to compute integral of surface heat fluxes and ocean heat content tendency:
# temp_Rate=xr.DataArray(temp_Rate)
temp_rate = temp_Rate.transpose('ocean_time','s_rho','eta_rho','xi_rho')
dT = dT.transpose('ocean_time','s_rho','eta_rho','xi_rho')

## Integrated heat tendency and sfc heat flux terms to check heat budget closure:

# 1. area-integral surface heat flux
tlen = len(temp_rate[:,0,0,0])
area_sum =  np.nansum(np.nansum(area,axis=1), axis=0)

shflux_int = np.empty((tlen))
for mm in np.arange(0,tlen):
    ###shflux_area = shflux[mm,:]*area*mask_shelf*mask_coast
    # needs xarray masking:
    shflux_area = shflux[mm,:].where(h < 1500)*area
    shflux_int[mm] = np.nansum(np.nansum(shflux_area,axis=1), axis=0)
    # del shflux_area
    
# 2. volume-integral heat tendency
temp_rate_int = np.empty((tlen))
temp_rate_vol = np.empty(np.squeeze(temp_Rate[:,0,:,:]).shape)
for mm in np.arange(0,tlen):
# - multplying by dz:
    temp_rate_dz = temp_Rate[mm,:].where(h < 1500)*dz[mm,:]
    temp_rate_vint = np.nansum(temp_rate_dz, axis=0)
    temp_rate_vol[mm,:] = temp_rate_vint*area*mask_shelf#*mask_coast
    del temp_rate_vint
    temp_rate_int[mm] = np.nansum(np.nansum(temp_rate_vol[mm,:],axis=1), axis=0)*Cp*rho0


In [ ]:
# %%time

# # 3. surface heat flux
# shflux2_int = np.empty((tlen))
# for mm in np.arange(0,tlen):
#     shflux2_area = shflux2[mm,:]*area*mask_shelf#*mask_coast
#     shflux2_int[mm] = np.nansum(np.nansum(shflux2_area,axis=1), axis=0)
#     del shflux2_area

# 4. volume-integral heat tendency using old temp_Rate2:
# temp_rate2_int = np.empty((tlen))
# temp_rate2_vol = np.empty(np.squeeze(temp_Rate2[:,0,:,:]).shape)
# for mm in np.arange(0,tlen):
# # - multplying by dz:
#     temp_rate2_dz = temp_Rate2[mm,:]*dz[mm,:]
#     temp_rate2_vint = np.nansum(temp_rate2_dz, axis=0)
#     temp_rate2_vol[mm,:] = temp_rate2_vint*area*mask_shelf
#     del temp_rate2_vint
#     temp_rate2_int[mm] = np.nansum(np.nansum(temp_rate2_vol[mm,:],axis=1), axis=0)*Cp*rho0


In [ ]:
    
fig, ax = plt.subplots(nrows=1, ncols=2, figsize = (12,9))

aa=ax[0,0].pcolormesh(np.nanmean(temp_rate_vol, axis=0)*mask_shelf, vmin=-100, vmax=100, cmap='coolwarm')
# plt.colorbar(aa)

bb=ax[0,1].pcolormesh(np.nanmean(shflux, axis=0)*mask_shelf, vmin=-100, vmax=100, cmap='coolwarm')



cax2 = plt.axes([0.92, 0.12, 0.01, 0.75])
cb = plt.colorbar(bb, cax=cax2, orientation='vertical')
cb.ax.set_ylabel('W.m$^{-2}$')

name_fig="waom2_OHC+shflux_shelf_annual_maps.png"
plt.savefig(fig_path + name_fig, dpi=300)
# plt.show()

In [ ]:
plt.plot(shflux[:,400,200])
plt.plot(shflux2[:,400,200],'--')

In [ ]:
# mask_shelf = ma.masked_where(h > 1500, np.ones(h.shape))

shflux_mm7 = shflux.isel(ocean_time=7).where(h < 1500)

In [ ]:
# plt.pcolormesh(shflux[7,:,:]*mask_shelf*mask_coast-shflux2[7,:,:]*mask_shelf*mask_coast, vmin=-100, vmax=100, cmap='coolwarm')
# plt.pcolormesh((shflux[7,:,:]*area*mask_shelf*mask_coast)-(shflux2[7,:,:]*area*mask_shelf*mask_coast), vmin=-100, vmax=100, cmap='coolwarm')
plt.pcolormesh(shflux[7,:,:].where(h < 1500), vmin=-100, vmax=100, cmap='coolwarm')
# plt.pcolormesh(shflux_mm7-(shflux2[7,:,:]*mask_shelf*mask_coast), vmin=-100, vmax=100, cmap='coolwarm')
# plt.pcolormesh(shflux[7,:,:]-(shflux2[7,:,:]*mask_shelf*mask_coast), vmin=-100, vmax=100, cmap='coolwarm')
# plt.pcolormesh(shflux[7,:,:].where(h < 1500)-(shflux2[7,:,:]*mask_shelf*mask_coast), vmin=-100, vmax=100, cmap='coolwarm')

plt.colorbar()

In [ ]:
# OHB integrated
print(np.nanmean(temp_rate_int))#*1e-15)
print(np.nanmean(shflux_int))#*1e-15)



In [ ]:
ohc_tend = temp_rate_int - np.nanmean(temp_rate_int)
shflux_tend = shflux_int - np.nanmean(shflux_int)

months=np.arange(0,73)*(5/30.41667)

plt.plot(months,shflux_tend, label='Sfc heat flux')
plt.plot(months,ohc_tend, label='OHC tendency')
plt.plot(months,shflux_int*0,'--k')
plt.plot(months,ohc_tend - shflux_tend, '--r', label='residual',linewidth=0.5)
plt.ylim([-1.5e14,1.5e14])

plt.legend()
plt.ylabel('Heat transport (W)')
plt.xlabel('Time (months)')
